In [1]:
# import modules

# system 
import re
import os

# calculation
import pandas as pd
import numpy as np

# plotting
%matplotlib inline
import seaborn
import matplotlib

# global stocktake tools
from gst_tools.make_plots import *
import gst_tools.gst_utils as utils


In [2]:
# USER INPUT

# First, choose which file you want to plot the data for
#data_file_name = 'UN-population-data-2017.csv'
data_file_name = 'PRIMAP-hist_v2.0_Energy-CO2.csv'
#data_file_name = 'PRIMAP-hist_UN-2017_calc__CO2-per-population.csv'
#data_file_name = 'WDI2017_GDP-PPP.csv'

# Second, choose which years you are interested in analysing
years_of_interest = ['1990', '2000', '2014']

In [3]:
# DATA READING AND PREP

# read the data from file 
fname_in = os.path.join('proc-data', data_file_name)
data = pd.read_csv(fname_in)

# Check the data format
if not utils.verify_data_format(data):
    print('WARNING: The data is not correctly formatted! Please check before continuing!')

# extract the key information
variable = data['variable'].unique()[0]
unit = data['unit'].unique()[0]

# tidy up for next stesps
data_years = utils.set_countries_as_index(data)
data_years = data_years.dropna(axis=1, how='any')

# remove comment below to display the data
#data_years

In [7]:
# TEMP - function definitions!

def make_histogram_select(df, var, unit_, remove_outliers=False, save_plot=False, kTuk=3, selected_country=''):

    """
    This is based on the make_simple_histogram function but caters to data that
    contains both positive and negative values. For the GST, it's important to be
    able to see whether or not trends etc. are positive or negative and a symmetric
    binning approach is needed.

    To calculate the bin sizes, we use a couple of conditional rules based on the data
    available, including the max and min of the data and the number of data points.
    For most plots we are expecting around 200 countries, but could also be a few regions.

    TODO - the 'outlier' calculation is helpful to see some data better BUT need to be careful.
    Proposed solution is to make BOTH plots so that it's clear to the user when data has been
    removed.

    TODO - 'df' is actually a series -> better name?
    """

    # Check the data - needs to not be, for example, all zeros
    if len(df.unique()) == 1:
        print('---------')
        print('All values in the series are the same! Exiting plotting routine for ' + str(var))
        print('---------')
        return

    # set a style
    sns.set(style="darkgrid")

    if selected_country:

        # get value of that country
        country_value = df[selected_country]
    
    if remove_outliers:
        # Outliers - in some cases, the date contains extreme outliers. These make for an unreadable
        # plot and in most cases arise from exceptional circumstances. These outliers are therefore removed
        # from the plots and the removal signalled to the user.
        # Example: Equatorial Guinea's emissions rose dramatically in the mid-90s due to the discovery of
        # oil. So much so, that the current emissions relative to 1990 are over 6000% higher. Including these
        # emissions in the plots would render a useless graph so we remove this country from the overview.

        # Use Tukey's fences and the interquartile range to set the bounds of the data
        # https://en.wikipedia.org/wiki/Outlier
        # For reference: kTUk default is set to 3 (above)
        # k = 1.5 -> outlier; k = 3 -> far out
        # TODO - get full and proper reference for this!!!

        print('-----------')
        print('Identifying and removing outliers')

        # calculate limits
        q75, q25 = np.percentile(df, [75, 25])
        iqr = q75 - q25
        tukey_min = q25 - kTuk * iqr
        tukey_max = q75 + kTuk * iqr
        # for testing:
        # print('tukey_min is ' + str(tukey_min))
        # print('tukey_max is ' + str(tukey_max))

        # Tell the user what the outliers are:
        lower_outliers = df[df < tukey_min]
        print('lower outliers are:')
        print(lower_outliers)
        upper_outliers = df[df > tukey_max]
        print('upper outliers are: ')
        print(upper_outliers)
        print('---')

        # actually remove the outliers
        df = df[(df > tukey_min) & (df < tukey_max)]

    # STATS
    # get some basic info about the data to use for setting styles, calculating bin sizes, and annotating plot
    maximum = max(df)
    minimum = min(df)
    mean = np.mean(df)
    median = np.median(df)
    npts = len(df)

    # Use data metrics to determine which approach to use for bins.
    if (minimum < 0) & (maximum > 0):

        # If both positive and negative, bins should be symmetric around 0!
        # What's the range of data?
        full_range = np.ceil(maximum - minimum)

        # Freedman–Diaconis rule
        # (need to recalculate IQR)
        q75, q25 = np.percentile(df, [75, 25])
        iqr = q75 - q25
        bin_width = int(2 * (iqr) / (npts ** (1 / 3)))

        # or the simple 'excel' rule:
        # bin_width = int(full_range / np.ceil(npts**(0.5)))

        # for nbins, need to take into account asymmetric distribution around 0
        nbins = int(np.ceil(2 * max([abs(minimum), abs(maximum)])) / bin_width)
        if not (nbins / 2).is_integer():
            nbins = nbins + 1

        # determine bin edges
        bins_calc = range(int((0 - (1 + nbins / 2) * bin_width)), int((0 + (1 + nbins / 2) * bin_width)), bin_width)
        print('bins set to ' + str(bins_calc))

    else:
        # use inbuilt Freedman-Diaconis
        # ? TODO - modify to ensure integers? or replicate above?
        bins_calc = 'fd'

    # --------------
    # MAKE THE PLOT

    # set up the figure
    fig, axs = plt.subplots()

    # make histogram
    sns.distplot(df,
                 kde=False,
                 bins=bins_calc,
                 color='mediumseagreen',
                 rug=False,
                 rug_kws={"color": "rebeccapurple", "alpha": 0.7, "linewidth": 0.4, "height": 0.03})

    # get xlims
    xmin, xmax = axs.get_xlim()
    
    # Dynamically set x axis range to make symmetric abut 0
    if minimum < 0:
        # reset xmin or xmax
        
        if np.absolute(xmax) > np.absolute(xmin):
            plt.xlim(-xmax, xmax)
        else:
            plt.xlim(xmin, -xmin)

        # and add a line at 0
        axs.axvline(linewidth=1, color='k')

    # If a country is selected for highlighting, then indicate it on the plot!
    if selected_country:

        # get value of that country
        #country_value = df[selected_country]

        if (country_value > xmin) & (country_value < xmax):
            # indicate it on the plot
            axs.axvline(x=country_value, linewidth=2, color='rebeccapurple')

            # annotate with country name
            ymin, ymax = axs.get_ylim()
            ypos = 0.65 * ymax
            axs.annotate(('  ' + to_name(selected_country) + ' ' + "{:.2g}".format(country_value)),
                     xy=(country_value, ypos), xycoords='data',
                     fontsize=9, color='rebeccapurple',
                     bbox=dict(facecolor='white', alpha=0.75),
                     )
            
        else:
            axs.annotate(('  ' + to_name(selected_country) + ' ' + "{:.2g}".format(country_value)),
                     xy=(.75, .65), xycoords=axs.transAxes,
                     fontsize=9, color='rebeccapurple',
                     bbox=dict(facecolor='white', alpha=0.75)
                     ) 

    # Annotate the plot with stats
    axs.annotate((" max = {:.2f}".format(maximum) +
                  "\n min = {:.2f}".format(minimum) +
                  "\n mean = {:.2f}".format(mean) +
                  "\n median = {:.2f}".format(median) +
                  "\n n = {:.0f}".format(npts)
                  ),
                 xy=(.75, 0.75), xycoords=axs.transAxes,
                 fontsize=9, color='black',
                 bbox=dict(facecolor='white', alpha=0.75))

    # label axes and add title
    axs.set_xlabel((var + ' (' + unit_ + ')'))
    axs.set_ylabel('Number of countries')
    axs.set_title((var + ' in ' + df.name), fontweight='bold')

    # save to file
    if save_plot:
        filepath = os.path.join('output', 'plots')
        fname = ('basic_histogram-' + var + '.pdf')
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        filename = os.path.join(filepath, fname)
        plt.savefig(filename, format='pdf', bbox_inches='tight')
        plt.close()

    # show the plot
    plt.show()

In [3]:
# Plot 1 - make a histogram of absolute data

for selected_year in years_of_interest:
    make_histogram(data_years[selected_year], variable, unit, 
                   remove_outliers=True, selected_country='VNM')


NameError: name 'make_histogram_select' is not defined

In [4]:
# Plot 2 - trends

# Calculate trends and define plotting params    
# TODO - improve description here. 
trends, rolling_trends, trends_unit = utils.calculate_trends(data_years, num_years_trend=5)
trends_variable = 'Annual average change in ' + variable

# plot the trend in the final year
make_histogram(rolling_trends.iloc[:,-1], trends_variable, 
               trends_unit, selected_country='VNM',
               save_plot=True)


Averaging trend over 5 years.
bins set to range(-38, 38, 2)


In [ ]:
df = rolling_trends['2014']
check = len(df[df < 0])
check
